<a href="https://colab.research.google.com/github/Pentuhov/PR.AI-2/blob/main/_%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B9_%D0%B1%D0%BB%D0%BE%D0%BA_%D0%9E%D0%B1%D1%83%D1%87%D0%B0%D1%8E%D1%89%D0%B0%D1%8F%2C_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B8_%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8_%D0%9F%D0%B5%D1%80%D0%B5%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9D%D0%A1_%D0%94%D0%97_Ultra_Pro_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используя базу "Пассажиры автобуса", подготовьте данные для обучения нейронной сети, классифицирующей изображение на два класса:
- входящий пассажир
- выходящий пассажир

Добейтесь точности работы модели на проверочной выборке не ниже 85%

Ссылка на датасет: https://storage.yandexcloud.net/aiueducation/Content/base/l4/bus.zip



Информацию о загрузке изображений и преобразовании их в numnpy-массивы вы можете найти в Базе знаний:

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gdown
import os
import pdb
import zipfile
from PIL import Image
%matplotlib inline

In [2]:
if not os.path.exists('/content/Входящий') or not os.path.exists('/content/Выходящий'):

  gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l4/bus.zip', None, quiet = True)

  with zipfile.ZipFile('bus.zip', 'r') as zip_file:
      zip_file.extractall('/content')

In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils

# Пути к каталогам с изображениями
input_dir = '/content/Входящий'
output_dir = '/content/Выходящий'

# Функция для загрузки и предобработки изображений
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img = Image.open(os.path.join(directory, filename))
            img = img.resize((100, 100))  # Подгоняем размер изображения по необходимости
            img = np.array(img) / 255.0  # Нормализуем изображение
            images.append(img)
    return images

# Загрузка изображений и создание меток классов
input_images = load_images(input_dir)
output_images = load_images(output_dir)
X = np.array(input_images + output_images)
y = np.array([0] * len(input_images) + [1] * len(output_images))  # 0 для входящего, 1 для выходящего

# Разделение данных на обучающий, проверочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Нормализация
X_train_normalized = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
X_val_normalized = X_val.reshape(X_val.shape[0], -1).astype('float32') / 255.0
X_test_normalized = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0
y_train_categorical = utils.to_categorical(y_train, 2)
y_val_categorical = utils.to_categorical(y_val, 2)
y_test_categorical = utils.to_categorical(y_test, 2)

# Создание модели
def create_model(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])
    return model

# Создание и компиляция модели
model = create_model(input_shape=X_train_normalized.shape[1:])
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model.fit(
    X_train_normalized,
    y_train_categorical,
    validation_data=(X_val_normalized, y_val_categorical),
    epochs=30,
    batch_size=32
)

# Оценка точности на обучающем, проверочном и тестовом наборах
train_loss, train_accuracy = model.evaluate(X_train_normalized, y_train_categorical, verbose=0)
val_loss, val_accuracy = model.evaluate(X_val_normalized, y_val_categorical, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test_categorical, verbose=0)

print("Точность на обучающем наборе: {:.2f}%".format(train_accuracy * 100))
print("Точность на проверочном наборе: {:.2f}%".format(val_accuracy * 100))
print("Точность на тестовом наборе: {:.2f}%".format(test_accuracy * 100))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 19s 88ms/step - accuracy: 0.5878 - loss: 0.9357 - val_accuracy: 0.7226 - val_loss: 0.5632
Epoch 2/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - accuracy: 0.7169 - loss: 0.6012 - val_accuracy: 0.7199 - val_loss: 0.5294
Epoch 3/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 21s 64ms/step - accuracy: 0.7614 - loss: 0.5215 - val_accuracy: 0.7977 - val_loss: 0.4643
Epoch 4/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - accuracy: 0.7831 - loss: 0.4721 - val_accuracy: 0.8390 - val_loss: 0.4176
Epoch 5/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 68ms/step - accuracy: 0.8009 - loss: 0.4360 - val_accuracy: 0.7213 - val_loss: 0.5585
Epoch 6/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.8128 - loss: 0.4187 - val_accuracy: 0.8396 - val_loss: 0.3853
Epoch 7/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.8200 - loss: 0.4089 - val_accuracy: 0.8211 - val_loss: 0.4390
Epoch 8/30
182/182 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.8159 - loss: 0.4029 - 